In [1]:
'''
    图像辨识: 语义分割、目标检测、定位、实例分割

    目标检测要同时识别对象的类别和位置:
    1.分类(Classification): 辨识对象的类别
    2,回归(Regression):找到对象的位置,包括对象左上角的坐标和宽度/高度
'''
# 滑动窗口算法
'''
    裁剪窗口 ---> 辨识窗口内是否有对象 --->滑动窗口 -->缩小原图尺寸 --->回到步骤1,辨识大尺寸对象
'''
import cv2
import time
import imutils

# 1.定义金字塔操作函数
## 参数: 原图, 每次缩小倍数, minSize:最小尺寸
def pyramid(image, scale=1.5, minSize=(30, 30)):

    yield image
    while True:
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        yield image
# 2.滑动窗口函数
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize): #向下滑动stepSize格
        for x in range(0, image.shape[1], stepSize): #向右滑动stepSize格
            # 传回裁剪后的窗口
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

# 4.测试
image = cv2.imread("./img/1.jpg")

## 窗口尺寸
winW,winH = 128,128

## 获取金字塔的各种尺寸
for resized in pyramid(image, scale=1.5):
    ### 滑动窗口
    for (x,y,window) in sliding_window(resized, stepSize=32, windowSize=(winW,winH)):
        # 窗口尺寸不合适即放弃,滑动到边缘时,尺寸过小
        if window.shape[0] != winH or window.shape[1] != winW:
            continue
        # 标示滑动的窗口
        clone = resized.copy()
        cv2.rectangle(clone, (x,y), (x+winW,y+winH), (0,255,0),2)
        cv2.imshow("Window", clone)
        cv2.waitKey(1)

        time.sleep(0.025)

# 结束时关闭窗口
cv2.destroyAllWindows()